In [1]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List
from utils import *

import numpy as np
import torch
import torch.nn as nn
from torch.optim import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from tqdm.auto import tqdm
import pickle

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
assert torch.cuda.is_available(), \
"CUDA support is not available."

In [3]:
device = torch.device("cuda:1")

In [4]:
dataloader = get_dataloader('cifar10', 256)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = VGG16().to(device)
model.load_state_dict(torch.load('./data/LC1_full_dataset.pth'))

<All keys matched successfully>

### Before collapse

In [6]:
evaluate_model(model, dataloader, count_nonzero_only=False, device=device)

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::prelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=91.24%
model has train accuracy=99.87%
model has size=513.37 MiB
model has macs=15.48 Gmacs
average inference time is 0.0026 seconds
model has 134.58 M parameters


In [7]:
model

VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stri

In [8]:
W0 = model._modules['fc'][0].weight.data
B0 = model._modules['fc'][0].bias.data
gamma0 = model._modules['fc'][1].weight.data
beta0 = model._modules['fc'][1].bias.data
mean0 = model._modules['fc'][1].running_mean
var0 = model._modules['fc'][1].running_var
eps = model._modules['fc'][1].eps
W1 = model._modules['fc1'][0].weight.data
B1 = model._modules['fc1'][0].bias.data

In [9]:
new_W0 = W1 @ torch.diag(gamma0 / torch.sqrt(var0 + eps)) @ W0
new_B0 = W1 @ (gamma0 * (B0 - mean0) / torch.sqrt(var0 + eps) + beta0) + B1

In [10]:
model._modules['fc'][0].weight.data = new_W0
model._modules['fc'][0].bias.data = new_B0
model._modules['fc'][1] = nn.Identity()
model._modules['fc'][2] = nn.Identity()
model._modules['fc1'][0] = nn.Identity()

### After collapse 1

In [11]:
evaluate_model(model, dataloader, count_nonzero_only=False, device=device)

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::prelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=91.29%
model has train accuracy=99.89%
model has size=449.32 MiB
model has macs=15.46 Gmacs
average inference time is 0.0025 seconds
model has 117.79 M parameters


In [12]:
model

VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stri

In [13]:
W0 = model._modules['fc'][0].weight.data
B0 = model._modules['fc'][0].bias.data
gamma1 = model._modules['fc1'][1].weight.data
beta1 = model._modules['fc1'][1].bias.data
mean1 = model._modules['fc1'][1].running_mean
var1 = model._modules['fc1'][1].running_var
eps = model._modules['fc1'][1].eps
W2 = model._modules['fc2'][0].weight.data
B2 = model._modules['fc2'][0].bias.data

In [14]:
new_W0 = W2 @ torch.diag(gamma1 / torch.sqrt(var1 + eps)) @ W0
new_B0 = W2 @ (gamma1 * (B0 - mean1) / torch.sqrt(var1 + eps) + beta1) + B2

In [32]:
model._modules['fc'][0] = nn.Linear(25088, 10).to(device)
model._modules['fc'][0].weight.data = new_W0
model._modules['fc'][0].bias.data = new_B0
model._modules['fc1'][1] = nn.Identity()
model._modules['fc1'][2] = nn.Identity()
model._modules['fc1'][3] = nn.Identity()
model._modules['fc2'][0] = nn.Identity()
model._modules['fc_small'] = nn.Identity()

In [33]:
evaluate(model, dataloader['val'], device=device)

91.67999267578125

In [34]:
evaluate_model(model, dataloader, count_nonzero_only=False, device=device)

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=91.75%
model has train accuracy=99.90%
model has size=57.12 MiB
model has macs=15.36 Gmacs
average inference time is 0.0019 seconds
model has 14.97 M parameters


In [28]:
model

VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stri

In [29]:
model._modules['fc_small'] = nn.Identity()